In [1]:
import os
import datetime as dt
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup 
import seaborn as sns 
import matplotlib.pyplot as plt

### Import and clean data from 2016 to 2020

In [2]:
def kml_extract(file, element, enconding='utf-8'):
    with open(file, 'r', encoding=enconding) as f:
        soup = BeautifulSoup(f) 
    return soup.findAll(element)

In [3]:
def nan_split(x): 
    if str(x) != 'nan' and len(str(x).split(' ')) == 2: 
        return str(x).split(' ')[1] 
    else: 
        return x

In [4]:
data_path = os.path.join(os.path.abspath(os.curdir), 'data')

In [5]:
data_2016 = kml_extract(os.path.join(data_path, 'ardida_2016.kml'), 'schemadata')

In [6]:
raw = []
year = []
code = []
region = [] 
place = [] 
start = [] 
end = []
total_ba = [] 
cause = []
for i in data_2016: 
    raw.append(i.get_text().split('\n')) 

for item in raw: 
    code.append(item[1]) 
    cause.append(item[4])
    region.append(item[5]) 
    place.append(item[6])
    start.append(item[9])
    end.append(item[10])  
    year.append(item[11])
    total_ba.append(item[12]) 
    
df_2016 = pd.DataFrame({'year': year, 
                         'code':code, 
                         'region':region, 
                         'place': place, 
                         'start': start, 
                         'end': end, 
                         'total_ba': total_ba, 
                         'cause': cause
                        }) 

In [7]:
df_2016

,year,code,region,place,start,end,total_ba,cause
0,2016.00000000,AT1161118,Évora,Borba,2016-12-07 12:10:00.000,2016-12-07 14:50:00.000,0.18381659,Negligente
1,2016.00000000,AT116190,Évora,Reguengos de Monsaraz,2016-06-06 14:12:00.000,2016-06-06 15:00:00.000,0.07917451,Desconhecida
2,2016.00000000,AT116191,Évora,Montemor-o-Novo,2016-06-07 14:47:00.000,2016-06-07 16:10:00.000,0.06372405,Negligente
3,2016.00000000,AT116274,Évora,Alandroal,2016-06-17 17:47:00.000,2016-06-17 21:45:00.000,13.16301499,Negligente
4,2016.00000000,AT116291,Évora,Montemor-o-Novo,2016-06-22 17:57:00.000,2016-06-22 20:35:00.000,1.51998117,Desconhecida
...,...,...,...,...,...,...,...,...
2840,71.68798655,DM2161944,,,,2016.00000000,,
2841,0.31812498,DM216887,,,,2016.00000000,,
2842,0.52987182,DM2161035,,,,2016.00000000,,
2843,0.34881254,DM2161260,,,,2016.00000000,,


In [8]:
df_2016.iloc[df_2016[df_2016['year'] != '2016.00000000'].index, 6] = df_2016.iloc[df_2016[df_2016['year'] != '2016.00000000'].index, 0]
df_2016.iloc[df_2016[df_2016['year'] != '2016.00000000'].index, 2:6] = np.NaN 
df_2016.iloc[df_2016[df_2016['year'] != '2016.00000000'].index, 7] = np.NaN 
df_2016.iloc[df_2016[df_2016['year'] != '2016.00000000'].index, 0] = '2016.00000000' 
df_2016['year'] = df_2016['year'].astype('float32').astype('int32')
df_2016['total_ba'] = df_2016['total_ba'].astype('float32')
df_2016['start_date'] = df_2016['start'].apply(lambda x: str(x).split(' ')[0]) 
df_2016['start_time'] = df_2016['start'].apply(lambda x: nan_split(x))
df_2016['end_date'] = df_2016['end'].apply(lambda x: str(x).split(' ')[0]) 
df_2016['end_time'] = df_2016['end'].apply(lambda x: nan_split(x))  
df_2016.drop('start', axis=1, inplace=True) 
df_2016.drop('end', axis=1, inplace=True) 
df_2016['end_time'] = df_2016['end_time'].apply(lambda x: str(x).split('.')[0]) 
df_2016['start_time'] = df_2016['start_time'].apply(lambda x: str(x).split('.')[0]) 
df_2016.iloc[df_2016[df_2016['start_date'] == 'nan'].index, 6] = np.NaN  
df_2016.iloc[df_2016[df_2016['start_time'] == 'nan'].index, 7] = np.NaN  
df_2016.iloc[df_2016[df_2016['end_date'] == 'nan'].index, -2] = np.NaN
df_2016.iloc[df_2016[df_2016['end_time'] == 'nan'].index, -1] = np.NaN  
df_2016 = pd.concat([df_2016.iloc[:, :4], df_2016.iloc[:, 6:10], df_2016.iloc[:, 4:6]], axis=1)

In [9]:
df_2016

,year,code,region,place,start_date,start_time,end_date,end_time,total_ba,cause
0,2016,AT1161118,Évora,Borba,2016-12-07,12:10:00,2016-12-07,14:50:00,0.183817,Negligente
1,2016,AT116190,Évora,Reguengos de Monsaraz,2016-06-06,14:12:00,2016-06-06,15:00:00,0.079175,Desconhecida
2,2016,AT116191,Évora,Montemor-o-Novo,2016-06-07,14:47:00,2016-06-07,16:10:00,0.063724,Negligente
3,2016,AT116274,Évora,Alandroal,2016-06-17,17:47:00,2016-06-17,21:45:00,13.163015,Negligente
4,2016,AT116291,Évora,Montemor-o-Novo,2016-06-22,17:57:00,2016-06-22,20:35:00,1.519981,Desconhecida
...,...,...,...,...,...,...,...,...,...,...
2840,2016,DM2161944,NaN,NaN,NaN,NaN,NaN,NaN,71.687988,NaN
2841,2016,DM216887,NaN,NaN,NaN,NaN,NaN,NaN,0.318125,NaN
2842,2016,DM2161035,NaN,NaN,NaN,NaN,NaN,NaN,0.529872,NaN
2843,2016,DM2161260,NaN,NaN,NaN,NaN,NaN,NaN,0.348813,NaN


In [10]:
df_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2845 entries, 0 to 2844
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        2845 non-null   int32  
 1   code        2845 non-null   object 
 2   region      2491 non-null   object 
 3   place       2491 non-null   object 
 4   start_date  2491 non-null   object 
 5   start_time  2491 non-null   object 
 6   end_date    2491 non-null   object 
 7   end_time    2491 non-null   object 
 8   total_ba    2845 non-null   float32
 9   cause       2491 non-null   object 
dtypes: float32(1), int32(1), object(8)
memory usage: 200.2+ KB


In [11]:
data_2017 = kml_extract(os.path.join(data_path, 'ardida_2017.kml'), 'schemadata')

In [12]:
raw = []
year = []
code = []
region = [] 
place = [] 
start = [] 
end = []
total_ba = [] 
cause = []
for i in data_2017: 
    raw.append(i.get_text().split('\n')) 

for item in raw: 
    code.append(item[1]) 
    cause.append(item[3])
    region.append(item[4]) 
    place.append(item[5])
    start.append(item[8])
    end.append(item[9])  
    year.append(item[10])
    total_ba.append(item[11]) 
    
df_2017 = pd.DataFrame({'year': year, 
                         'code':code, 
                         'region':region, 
                         'place': place, 
                         'start': start, 
                         'end': end, 
                         'total_ba': total_ba, 
                         'cause': cause
                        }) 

In [13]:
df_2017

,year,code,region,place,start,end,total_ba,cause
0,2017,AT117500,Évora,Évora,2017-06-18 16:50:00.000,2017-06-19 07:50:00.000,3.06536660,Negligente
1,2017,AG117147,Faro,Lagoa,2017-05-10 12:50:00.000,2017-05-10 13:47:00.000,0.08873183,Desconhecida
2,2017,AG117200,Faro,Silves,2017-05-31 17:38:00.000,2017-05-31 18:50:00.000,0.25311423,Negligente
3,2017,AG117203,Faro,Albufeira,2017-06-02 14:58:00.000,2017-06-02 19:05:00.000,4.69093393,Negligente
4,2017,AG117209,Faro,Lagoa,2017-06-04 03:37:00.000,2017-06-04 04:44:00.000,1.66958362,Desconhecida
...,...,...,...,...,...,...,...,...
2794,2017,DM3174767,Viana do Castelo,Arcos de Valdevez,2017-10-15 15:21:00.000,2017-10-16 20:15:00.000,276.76470000,Intencional
2795,2017,DM3174334,Viana do Castelo,Arcos de Valdevez,2017-08-21 16:58:00.000,2017-08-22 13:59:00.000,66.77240000,Intencional
2796,2017,BL4172371,Leiria,Alcobaça,2017-10-15 14:33:00.000,2017-10-18 20:10:00.000,,Reacendimento
2797,2017,BL4172366,Leiria,Alcobaça,2017-10-15 13:51:00.000,2017-10-17 10:48:00.000,,Reacendimento


In [14]:
df_2017.drop(index=2798, inplace=True) 
df_2017.iloc[df_2017[df_2017['total_ba'] == ''].index, -2] = np.NaN 
df_2017['total_ba'] = df_2017['total_ba'].astype('float32') 
df_2017['year'] = df_2017['year'].astype('float32').astype('int32') 
df_2017['start_date'] = df_2017['start'].apply(lambda x: str(x).split(' ')[0]) 
df_2017['start_time'] = df_2017['start'].apply(lambda x: nan_split(x))
df_2017['end_date'] = df_2017['end'].apply(lambda x: str(x).split(' ')[0]) 
df_2017['end_time'] = df_2017['end'].apply(lambda x: nan_split(x)) 
df_2017.drop('start', axis=1, inplace=True) 
df_2017.drop('end', axis=1, inplace=True)  
df_2017['end_time'] = df_2017['end_time'].apply(lambda x: str(x).split('.')[0]) 
df_2017['start_time'] = df_2017['start_time'].apply(lambda x: str(x).split('.')[0])  
df_2017 = pd.concat([df_2017.iloc[:, :4], df_2017.iloc[:, 6:10], df_2017.iloc[:, 4:6]], axis=1)

In [15]:
df_2017

,year,code,region,place,start_date,start_time,end_date,end_time,total_ba,cause
0,2017,AT117500,Évora,Évora,2017-06-18,16:50:00,2017-06-19,07:50:00,3.065367,Negligente
1,2017,AG117147,Faro,Lagoa,2017-05-10,12:50:00,2017-05-10,13:47:00,0.088732,Desconhecida
2,2017,AG117200,Faro,Silves,2017-05-31,17:38:00,2017-05-31,18:50:00,0.253114,Negligente
3,2017,AG117203,Faro,Albufeira,2017-06-02,14:58:00,2017-06-02,19:05:00,4.690934,Negligente
4,2017,AG117209,Faro,Lagoa,2017-06-04,03:37:00,2017-06-04,04:44:00,1.669584,Desconhecida
...,...,...,...,...,...,...,...,...,...,...
2793,2017,BI2173957,Castelo Branco,Sertã,2017-07-23,13:47:00,2017-09-20,09:48:00,33711.753906,Intencional
2794,2017,DM3174767,Viana do Castelo,Arcos de Valdevez,2017-10-15,15:21:00,2017-10-16,20:15:00,276.764709,Intencional
2795,2017,DM3174334,Viana do Castelo,Arcos de Valdevez,2017-08-21,16:58:00,2017-08-22,13:59:00,66.772400,Intencional
2796,2017,BL4172371,Leiria,Alcobaça,2017-10-15,14:33:00,2017-10-18,20:10:00,NaN,Reacendimento


In [16]:
df_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2798 entries, 0 to 2797
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        2798 non-null   int32  
 1   code        2798 non-null   object 
 2   region      2798 non-null   object 
 3   place       2798 non-null   object 
 4   start_date  2798 non-null   object 
 5   start_time  2798 non-null   object 
 6   end_date    2798 non-null   object 
 7   end_time    2798 non-null   object 
 8   total_ba    2796 non-null   float32
 9   cause       2798 non-null   object 
dtypes: float32(1), int32(1), object(8)
memory usage: 218.6+ KB


In [17]:
data_2018 = kml_extract(os.path.join(data_path, 'ardida_2018.kml'), 'schemadata')

In [18]:
raw = []
year = []
code = []
region = [] 
place = [] 
start = [] 
end = []
total_ba = [] 
cause = []
for i in data_2018: 
    raw.append(i.get_text().split('\n')) 

for item in raw: 
    if len(item) >= 14: 
        code.append(item[1]) 
        cause.append(item[4])
        region.append(item[5]) 
        place.append(item[6])
        start.append(item[9])
        end.append(item[10])  
        year.append(item[11])
        total_ba.append(item[12]) 
    else: 
        continue
    
df_2018 = pd.DataFrame({'year': year, 
                         'code':code, 
                         'region':region, 
                         'place': place, 
                         'start': start, 
                         'end': end, 
                         'total_ba': total_ba, 
                         'cause': cause
                        }) 

In [19]:
df_2018

,year,code,region,place,start,end,total_ba,cause
0,2018,DM4183094,Braga,Fafe,2018-08-26 11:44:00.000,2018-08-27 00:50:00.000,18.49743029,Desconhecida
1,2018,DM4182947,Braga,Guimarães,2018-08-15 15:05:00.000,2018-08-16 03:40:00.000,15.39442973,Desconhecida
2,2018,AG118728,Faro,Monchique,2018-08-03 13:32:00.000,2018-09-23 17:39:00.000,26885.44880657,Desconhecida
3,2018,TM1181418,Vila Real,Alijó,2018-09-23 21:04:00.000,2018-09-28 19:50:00.000,474.57879232,Negligente
4,2018,RO2182659,Lisboa,Cascais,2018-10-06 22:50:00.000,2018-10-11 19:30:00.000,428.97535349,Desconhecida
...,...,...,...,...,...,...,...,...
503,2018,BL21836,Coimbra,Pampilhosa da Serra,2018-02-08 10:45:00.000,2018-02-08 14:50:00.000,0.99956630,Negligente
504,2018,BL3181904,Aveiro,Águeda,2018-09-02 14:15:00.000,2018-09-02 17:20:00.000,1.58550254,Desconhecida
505,2018,BL3181961,Aveiro,Águeda,2018-09-10 04:44:00.000,2018-09-10 08:09:00.000,1.06982767,Desconhecida
506,2018,BL3181966,Aveiro,Águeda,2018-09-10 14:33:00.000,2018-09-10 20:09:00.000,2.31164446,Reacendimento


In [20]:
df_2018['total_ba'] = df_2018['total_ba'].astype('float32') 
df_2018['year'] = df_2018['year'].astype('float32').astype('int32')  
df_2018['start_date'] = df_2018['start'].apply(lambda x: str(x).split(' ')[0]) 
df_2018['start_time'] = df_2018['start'].apply(lambda x: nan_split(x))
df_2018['end_date'] = df_2018['end'].apply(lambda x: str(x).split(' ')[0]) 
df_2018['end_time'] = df_2018['end'].apply(lambda x: nan_split(x))  
df_2018.drop('start', axis=1, inplace=True) 
df_2018.drop('end', axis=1, inplace=True) 
df_2018['end_time'] = df_2018['end_time'].apply(lambda x: str(x).split('.')[0]) 
df_2018['start_time'] = df_2018['start_time'].apply(lambda x: str(x).split('.')[0])  
df_2018 = pd.concat([df_2018.iloc[:, :4], df_2018.iloc[:, 6:10], df_2018.iloc[:, 4:6]], axis=1)

In [21]:
df_2018

,year,code,region,place,start_date,start_time,end_date,end_time,total_ba,cause
0,2018,DM4183094,Braga,Fafe,2018-08-26,11:44:00,2018-08-27,00:50:00,18.497431,Desconhecida
1,2018,DM4182947,Braga,Guimarães,2018-08-15,15:05:00,2018-08-16,03:40:00,15.394430,Desconhecida
2,2018,AG118728,Faro,Monchique,2018-08-03,13:32:00,2018-09-23,17:39:00,26885.449219,Desconhecida
3,2018,TM1181418,Vila Real,Alijó,2018-09-23,21:04:00,2018-09-28,19:50:00,474.578796,Negligente
4,2018,RO2182659,Lisboa,Cascais,2018-10-06,22:50:00,2018-10-11,19:30:00,428.975342,Desconhecida
...,...,...,...,...,...,...,...,...,...,...
503,2018,BL21836,Coimbra,Pampilhosa da Serra,2018-02-08,10:45:00,2018-02-08,14:50:00,0.999566,Negligente
504,2018,BL3181904,Aveiro,Águeda,2018-09-02,14:15:00,2018-09-02,17:20:00,1.585503,Desconhecida
505,2018,BL3181961,Aveiro,Águeda,2018-09-10,04:44:00,2018-09-10,08:09:00,1.069828,Desconhecida
506,2018,BL3181966,Aveiro,Águeda,2018-09-10,14:33:00,2018-09-10,20:09:00,2.311645,Reacendimento


In [22]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        508 non-null    int32  
 1   code        508 non-null    object 
 2   region      508 non-null    object 
 3   place       508 non-null    object 
 4   start_date  508 non-null    object 
 5   start_time  508 non-null    object 
 6   end_date    508 non-null    object 
 7   end_time    508 non-null    object 
 8   total_ba    508 non-null    float32
 9   cause       508 non-null    object 
dtypes: float32(1), int32(1), object(8)
memory usage: 35.8+ KB


In [23]:
data_2019 = kml_extract(os.path.join(data_path, 'ardida_2019.kml'), 'schemadata')

In [24]:
raw = []
year = []
code = []
start = [] 
end = []
total_ba = [] 
cause = []
for i in data_2019: 
    raw.append(i.get_text().split('\n')) 

for item in raw:  
    code.append(item[1]) 
    cause.append(item[3])
    start.append(item[4])
    end.append(item[5])  
    year.append(item[6])
    total_ba.append(item[7]) 

    
df_2019 = pd.DataFrame({'year': year, 
                         'code':code, 
                         'start': start, 
                         'end': end, 
                         'total_ba': total_ba, 
                         'cause': cause
                        }) 

In [25]:
df_2019

,year,code,start,end,total_ba,cause
0,2019,AT119105,2019-05-02 16:11:00.000,2019-05-02 16:47:00.000,0.19790280,Negligente
1,2019,AT119110,2019-05-07 13:17:00.000,2019-05-07 13:49:00.000,0.03758401,Negligente
2,2019,AT119118,2019-05-13 15:55:00.000,2019-05-13 17:04:00.000,2.58176485,Desconhecida
3,2019,AT119124,2019-05-14 17:00:00.000,2019-05-14 17:50:00.000,1.59100140,Negligente
4,2019,AT119127,2019-05-15 18:20:00.000,2019-05-15 22:23:00.000,3.50104192,Desconhecida
...,...,...,...,...,...,...
1724,2019,,,,0.02963959,
1725,2019,,,,0.01955740,
1726,2019,,,,10.80956728,
1727,2019,,,,16.12766853,


In [26]:
df_2019['total_ba'] = df_2019['total_ba'].astype('float32') 
df_2019['year'] = df_2019['year'].astype('float32').astype('int32')  
df_2019.iloc[df_2019[df_2019['code'] == ' '].index, 1:4] = np.NaN 
df_2019.iloc[df_2019[df_2019['cause'] == ' '].index, -1] = np.NaN 
df_2019['start_date'] = df_2019['start'].apply(lambda x: str(x).split(' ')[0]) 
df_2019['start_time'] = df_2019['start'].apply(lambda x: nan_split(x))
df_2019['end_date'] = df_2019['end'].apply(lambda x: str(x).split(' ')[0]) 
df_2019['end_time'] = df_2019['end'].apply(lambda x: nan_split(x))  
df_2019.drop('start', axis=1, inplace=True) 
df_2019.drop('end', axis=1, inplace=True)  
df_2019['region'] = np.NaN 
df_2019['place'] = np.NaN 
df_2019['end_time'] = df_2019['end_time'].apply(lambda x: str(x).split('.')[0]) 
df_2019['start_time'] = df_2019['start_time'].apply(lambda x: str(x).split('.')[0])  
df_2019 = pd.concat([df_2019.iloc[:, :2], df_2019.iloc[:, -2:], df_2019.iloc[:,4:8], df_2019.iloc[:, 2:4]], axis=1) 
df_2019.iloc[df_2019[df_2019['start_date'] == 'nan'].index, 4] = np.NaN  
df_2019.iloc[df_2019[df_2019['start_time'] == 'nan'].index, 5] = np.NaN  
df_2019.iloc[df_2019[df_2019['end_date'] == 'nan'].index, 6] = np.NaN  
df_2019.iloc[df_2019[df_2019['end_time'] == 'nan'].index, 7] = np.NaN 

In [27]:
df_2019

,year,code,region,place,start_date,start_time,end_date,end_time,total_ba,cause
0,2019,AT119105,NaN,NaN,2019-05-02,16:11:00,2019-05-02,16:47:00,0.197903,Negligente
1,2019,AT119110,NaN,NaN,2019-05-07,13:17:00,2019-05-07,13:49:00,0.037584,Negligente
2,2019,AT119118,NaN,NaN,2019-05-13,15:55:00,2019-05-13,17:04:00,2.581765,Desconhecida
3,2019,AT119124,NaN,NaN,2019-05-14,17:00:00,2019-05-14,17:50:00,1.591001,Negligente
4,2019,AT119127,NaN,NaN,2019-05-15,18:20:00,2019-05-15,22:23:00,3.501042,Desconhecida
...,...,...,...,...,...,...,...,...,...,...
1724,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.029640,NaN
1725,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.019557,NaN
1726,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.809567,NaN
1727,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.127668,NaN


In [28]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1729 entries, 0 to 1728
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        1729 non-null   int32  
 1   code        1720 non-null   object 
 2   region      0 non-null      float64
 3   place       0 non-null      float64
 4   start_date  1720 non-null   object 
 5   start_time  1720 non-null   object 
 6   end_date    1720 non-null   object 
 7   end_time    1720 non-null   object 
 8   total_ba    1729 non-null   float32
 9   cause       1718 non-null   object 
dtypes: float32(1), float64(2), int32(1), object(6)
memory usage: 121.7+ KB


In [29]:
data_2020 = kml_extract(os.path.join(data_path, 'ardida_2020.kml'), 'schemadata')

In [30]:
raw = []
year = []
code = []
start = [] 
end = []
total_ba = [] 
cause = []
for i in data_2020: 
    raw.append(i.get_text().split('\n')) 

for item in raw:  
    code.append(item[1]) 
    cause.append(item[4])
    start.append(item[5])
    end.append(item[6])  
    year.append(item[7])
    total_ba.append(item[8]) 

    
df_2020 = pd.DataFrame({'year': year, 
                         'code':code, 
                         'start': start, 
                         'end': end, 
                         'total_ba': total_ba, 
                         'cause': cause
                        }) 

In [31]:
df_2020

,year,code,start,end,total_ba,cause
0,2020,BL220209,2020-07-05 13:24:00.000,2020-07-05 15:08:00.000,1.13659484,Intencional
1,2020,DM2201036,2020-07-05 16:36:00.000,2020-07-05 17:08:00.000,1.03651303,Reacendimento
2,2020,DM2201021,2020-07-05 08:56:00.000,2020-07-05 10:12:00.000,1.15395403,Desconhecida
3,2020,TM120183,2020-04-04 20:42:00.000,2020-04-05 02:00:00.000,2.85423500,Negligente
4,2020,TM120156,2020-03-29 20:20:00.000,2020-03-29 23:00:00.000,2.48005195,Negligente
...,...,...,...,...,...,...
1793,2020,AG120614,2020-07-06 14:56:00.000,2020-07-07 10:00:00.000,535.63297426,Negligente
1794,2020,BL220206,2020-07-04 13:09:00.000,2020-07-05 00:53:00.000,156.27918117,Intencional
1795,2020,AT320158,2020-07-04 16:00:00.000,2020-07-04 17:07:00.000,1.40719244,Desconhecida
1796,2020,RO120327,2020-07-05 16:09:00.000,2020-07-05 20:39:00.000,26.59060520,Reacendimento


In [32]:
df_2020['total_ba'] = df_2020['total_ba'].astype('float32')  
df_2020['year'] = df_2020['year'].astype('float32').astype('int32')   
df_2020['start_date'] = df_2020['start'].apply(lambda x: str(x).split(' ')[0]) 
df_2020['start_time'] = df_2020['start'].apply(lambda x: nan_split(x))
df_2020['end_date'] = df_2020['end'].apply(lambda x: str(x).split(' ')[0]) 
df_2020['end_time'] = df_2020['end'].apply(lambda x: nan_split(x))  
df_2020.drop('start', axis=1, inplace=True) 
df_2020.drop('end', axis=1, inplace=True) 
df_2020['region'] = np.NaN 
df_2020['place'] = np.NaN  
df_2020['end_time'] = df_2020['end_time'].apply(lambda x: str(x).split('.')[0]) 
df_2020['start_time'] = df_2020['start_time'].apply(lambda x: str(x).split('.')[0])  
df_2020 = pd.concat([df_2020.iloc[:, :2], df_2020.iloc[:, -2:], df_2020.iloc[:,4:8], df_2020.iloc[:, 2:4]], axis=1) 

In [33]:
df_2020

,year,code,region,place,start_date,start_time,end_date,end_time,total_ba,cause
0,2020,BL220209,NaN,NaN,2020-07-05,13:24:00,2020-07-05,15:08:00,1.136595,Intencional
1,2020,DM2201036,NaN,NaN,2020-07-05,16:36:00,2020-07-05,17:08:00,1.036513,Reacendimento
2,2020,DM2201021,NaN,NaN,2020-07-05,08:56:00,2020-07-05,10:12:00,1.153954,Desconhecida
3,2020,TM120183,NaN,NaN,2020-04-04,20:42:00,2020-04-05,02:00:00,2.854235,Negligente
4,2020,TM120156,NaN,NaN,2020-03-29,20:20:00,2020-03-29,23:00:00,2.480052,Negligente
...,...,...,...,...,...,...,...,...,...,...
1793,2020,AG120614,NaN,NaN,2020-07-06,14:56:00,2020-07-07,10:00:00,535.632996,Negligente
1794,2020,BL220206,NaN,NaN,2020-07-04,13:09:00,2020-07-05,00:53:00,156.279175,Intencional
1795,2020,AT320158,NaN,NaN,2020-07-04,16:00:00,2020-07-04,17:07:00,1.407192,Desconhecida
1796,2020,RO120327,NaN,NaN,2020-07-05,16:09:00,2020-07-05,20:39:00,26.590605,Reacendimento


In [34]:
df2 = pd.concat([df_2016, df_2017, df_2018, df_2019, df_2020], axis=0, ignore_index=True)  
df2.rename(columns={'place': 'county'}, inplace=True) 
df2['cause'] = df2['cause'].apply(lambda x: str(x).lower())  

### Import and clean original dataset

In [35]:
data_path = os.path.join(os.path.abspath(os.curdir), 'data', 'florestal_fires.csv')
df = pd.read_csv(data_path, sep=',', encoding='cp860', low_memory=False) 
df.head(5)

,Ano,Codigo,Distrito,Concelho,DataAlerta,HoraAlerta,DataExtincao,HoraExtincao,AA_Total(pov+mato+agric),Reacendimento,Causa,TipoCausa,Grupo_causa,Categoria_causa,Dias_extincao
0,2001.0,BL20190,Coimbra,Oliveira do Hospital,02/07/2001,16:40:00,02/07/2001,17:25:00,0.3,1.0,12.0,Negligente,Desconhecida ...,Desconhecida,0.0
1,2001.0,DM201114,Aveiro,Arouca,03/08/2001,17:47:00,06/08/2001,18:15:00,174.0,0.0,12.0,Negligente,Desconhecida ...,Desconhecida,3.0
2,2001.0,DM401117,Viana do Castelo,Melga?o,17/09/2001,18:59:00,18/09/2001,02:40:00,54.0,0.0,12.0,Negligente,Desconhecida ...,Desconhecida,1.0
3,2001.0,DM901112,Braga,Terras de Bouro,04/09/2001,23:05:00,05/09/2001,11:15:00,80.0,0.0,12.0,Negligente,Desconhecida ...,Desconhecida,1.0
4,2001.0,DM901113,Braga,Amares,06/09/2001,18:50:00,07/09/2001,00:25:00,28.0,0.0,12.0,Negligente,Desconhecida ...,Desconhecida,1.0


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321985 entries, 0 to 321984
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Ano                       321984 non-null  float64
 1   Codigo                    321984 non-null  object 
 2   Distrito                  321984 non-null  object 
 3   Concelho                  321984 non-null  object 
 4   DataAlerta                236236 non-null  object 
 5   HoraAlerta                321984 non-null  object 
 6   DataExtincao              236041 non-null  object 
 7   HoraExtincao              321660 non-null  object 
 8   AA_Total(pov+mato+agric)  321984 non-null  float64
 9   Reacendimento             321984 non-null  float64
 10  Causa                     107615 non-null  float64
 11  TipoCausa                 321984 non-null  object 
 12  Grupo_causa               101023 non-null  object 
 13  Categoria_causa           101023 non-null  o

In [37]:
df.isna().sum()

Ano                              1
Codigo                           1
Distrito                         1
Concelho                         1
DataAlerta                   85749
HoraAlerta                       1
DataExtincao                 85944
HoraExtincao                   325
AA_Total(pov+mato+agric)         1
Reacendimento                    1
Causa                       214370
TipoCausa                        1
Grupo_causa                 220962
Categoria_causa             220962
Dias_extincao                  440
dtype: int64

### Clean Ano

In [38]:
df[df['Ano'].isna()] 

,Ano,Codigo,Distrito,Concelho,DataAlerta,HoraAlerta,DataExtincao,HoraExtincao,AA_Total(pov+mato+agric),Reacendimento,Causa,TipoCausa,Grupo_causa,Categoria_causa,Dias_extincao
321984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [39]:
# delete row filled with missing values
df.drop(df[df['Ano'].isna()].index, inplace=True)

In [40]:
df.isna().sum()

Ano                              0
Codigo                           0
Distrito                         0
Concelho                         0
DataAlerta                   85748
HoraAlerta                       0
DataExtincao                 85943
HoraExtincao                   324
AA_Total(pov+mato+agric)         0
Reacendimento                    0
Causa                       214369
TipoCausa                        0
Grupo_causa                 220961
Categoria_causa             220961
Dias_extincao                  440
dtype: int64

### Clean Grupo_causa

In [41]:
df['Grupo_causa'].unique()

array(['Desconhecida                                             ',
       'Clandestinos                                             ',
       'Vandalismo                                               ',
       'Outras situaτ⌡es dolosas                                 ', nan,
       'Autßrquica                                               ',
       'Ind·stria                                                ',
       'Actividades clandestinas                                 ',
       'N·cleos habitacionais permanentes                        ',
       'N·cleos habitacionais temporßrios associados ao recreio  ',
       'Limpeza do solo agrφcola                                 ',
       'Limpeza do solo florestal                                ',
       'Limpeza de ßreas urbanizadas                             ',
       'Borralheiras                                             ',
       'Renovaτπo de pastagens                                   ',
       'Penetraτπo em ßreas de caτa e marge

In [42]:
df['Grupo_causa'] = df['Grupo_causa'].apply(lambda x: str(x).rstrip().lower())

In [43]:
raw = df['Grupo_causa'].unique()
causas = ['desconhecida', 'clandestinos', 'vandalismo',
       'outras situações dolosas', 'nan', 'autárquica', 'indústria',
       'actividades clandestinas', 'núcleos habitacionais permanentes',
       'núcleos habitacionais temporários associados ao recreio',
       'limpeza do solo agrícola', 'limpeza do solo florestal',
       'limpeza de áreas urbanizadas', 'borralheiras',
       'renovação de pastagens',
       'penetração em áreas de caça e margens dos rios',
       'limpeza de caminhos, acessos e instalações',
       'proteção contra incêndios', 'outras', 'com medidas preventivas',
       'auto-ignição', 'recreio e lazer', 'confecção de comida',
       'fumadores a pé', 'em circulação motorizada', 'fumigação',
       'linhas eléctricas', 'caminhos de ferro', 'tubos de escape',
       'acidentes de viação', 'alfaias agrícolas', 'máquinas agrícolas',
       'equipamento florestal', 'motosserras', 'máquinas florestais',
       'máquinas industriais', 'outra maquinaria e equipamento', 'vidros',
       'conflitos de caça', 'danos provocados pela vida selvagem',
       'alterações no uso do solo',
       'pressão para venda de material lenhoso',
       'limitação ao uso e gestão do solo',
       'contradições no uso e fruição dos baldios',
       'brincadeiras de crianτas', 'irresponsabilidade de menores',
       'piromania', 'outras situações inimputáveis',
       'manobras de diversão', 'provocação aos meios de combate',
       'conflitos entre vizinhos', 'vinganças', 'prova material',
       'prova pessoal', 'outras informações', 'reparação de estradas',
       'de habitação', 'outros acidentes', 'soldaduras', 'aquecimento',
       'exercícios militares', 'comércio', 'desinfestação', 'industriais',
       'explosivos', 'disparos de caçadores']

In [44]:
for i in range(len(causas)):
    df.iloc[df[df['Grupo_causa'] == raw[i]].index, 12] = causas[i]

In [45]:
df.iloc[df[df['Grupo_causa'] == 'nan'].index, 12] = np.NaN

In [46]:
df['Grupo_causa'].value_counts()

desconhecida              23603
vandalismo                18837
outras informações        15259
renovação de pastagens    11698
prova material             7095
                          ...  
de habitação                 13
explosivos                   10
industriais                   9
reparação de estradas         6
disparos de caçadores         3
Name: Grupo_causa, Length: 65, dtype: int64

### Clean TipoCausa

In [47]:
df['TipoCausa'].unique()

array(['Negligente  ', 'Intencional ', 'Natural     ', 'Desconhecida',
       'NULL        ', 'Reacendiment'], dtype=object)

In [48]:
df['TipoCausa'] = df['TipoCausa'].apply(lambda x: str(x).rstrip().lower()) 
df.iloc[df[df['TipoCausa'] == 'null'].index, 11] = np.NaN 
df.iloc[df[df['TipoCausa'] == 'reacendiment'].index, 11] = 'reacendimento'

In [49]:
df['TipoCausa'].value_counts()

desconhecida     39475
negligente       33827
intencional      26950
reacendimento     6402
natural            961
Name: TipoCausa, dtype: int64

### Clean Categoria_causa

In [50]:
df['Categoria_causa'].unique()

array(['Desconhecida  ', 'Uso do fogo   ', 'Incendiarismo ', nan,
       'Acidentais    ', 'Estruturais   ', 'Indeterminadas'], dtype=object)

In [51]:
df['Categoria_causa'] = df['Categoria_causa'].apply(lambda x: str(x).rstrip().lower()) 
df.iloc[df[df['Categoria_causa'] == 'nan'].index, -2] = np.NaN

In [52]:
df['Categoria_causa'].value_counts()

uso do fogo       28209
desconhecida      23603
incendiarismo     22730
indeterminadas    22644
acidentais         2880
estruturais         957
Name: Categoria_causa, dtype: int64

### Clean Distrito

In [53]:
df['Distrito'].unique()

array(['Coimbra                       ', 'Aveiro                        ',
       'Viana do Castelo              ', 'Braga                         ',
       'Vila Real                     ', '?vora                         ',
       'Set?bal                       ', 'Beja                          ',
       'Portalegre                    ', 'Guarda                        ',
       'Castelo Branco                ', 'Viseu                         ',
       'Leiria                        ', 'Porto                         ',
       'Lisboa                        ', 'Santar?m                      ',
       'Bragan?a                      ', 'Faro                          ',
       'BRAGA                         ', 'Viana Do Castelo              '],
      dtype=object)

In [54]:
df['Distrito'] = df['Distrito'].apply(lambda x: str(x).rstrip())  
df.iloc[df[df['Distrito'] == '?vora'].index, 2] = 'Évora' 
df.iloc[df[df['Distrito'] == 'Set?bal'].index, 2] = 'Setúbal'  
df.iloc[df[df['Distrito'] == 'Santar?m'].index, 2] = 'Santarém' 
df.iloc[df[df['Distrito'] == 'Bragan?a'].index, 2] = 'Bragança' 
df.iloc[df[df['Distrito'] == 'Viana Do Castelo'].index, 2] = 'Viana do Castelo'

In [55]:
df['Distrito'].value_counts()

Porto               77638
Braga               41611
Viseu               28116
Aveiro              27661
Lisboa              25273
Vila Real           24362
Viana do Castelo    21505
Guarda              12375
Bragança            11355
Setúbal              9835
Santarém             9619
Leiria               8869
Coimbra              8180
Castelo Branco       6794
Faro                 5595
Portalegre           1193
Beja                 1153
Évora                 849
BRAGA                   1
Name: Distrito, dtype: int64

### Clean Concelho

In [56]:
df['Categoria_causa'].unique()

array(['desconhecida', 'uso do fogo', 'incendiarismo', nan, 'acidentais',
       'estruturais', 'indeterminadas'], dtype=object)

In [57]:
df['Concelho'] = df['Concelho'].apply(lambda x: str(x).rstrip().lower())  

In [58]:
raw = df['Concelho'].unique()
concelhos = ['Oliveira do Hospital', 'Arouca', 'Melgaço', 'Terras de Bouro',
       'Amares', 'Viana do Castelo', 'Fafe', 'Vieira do Minho',
       'Ponte da Barca', 'Monção', 'Cabeceiras de Basto',
       'Ribeira de Pena', 'Mondim de Basto', 'Évora', 'Santiago do Cacém',
       'Almodóvar', 'Viana do Alentejo', 'Elvas', 'Mértola', 'Avis',
       'Aljustrel', 'Alcácer do Sal', 'Redondo', 'Beja', 'Grândola',
       'Crato', 'Trancoso', 'Covilhã', 'Oleiros', 'Sertã', 'Vila de Rei',
       'Fundão', 'Proença-a-Nova', 'Castro Daire', 'Sátão', 'Viseu',
       'Murtosa', 'Carregal do Sal', 'Anadia', 'Ovar',
       'Vila Nova de Paiva', 'Águeda', 'Mangualde', 'Aguiar da Beira',
       'Penalva do Castelo', 'Mealhada', 'Pombal', 'Penacova',
       'Pampilhosa da Serra', 'Coimbra', 'Vila Nova de Poiares',
       'Figueiró dos Vinhos', 'Lousã', 'Tábua', 'Miranda do Corvo',
       'Marinha Grande', 'Porto de Mós', 'Pedrógão Grande', 'Góis',
       'Arganil', 'Castanheira de Pêra', 'Ansião', 'Cantanhede', 'Leiria',
       'Mira', 'Penela', 'Ponte de Lima', 'Paredes de Coura', 'Amarante',
       'Marco de Canaveses', 'Vila Nova de Famalicão', 'Braga', 'Valença',
       'Santa Maria da Feira', 'Sintra', 'Cartaxo', 'Almeirim',
       'Azambuja', 'Abrantes', 'Salvaterra de Magos', 'Chamusca',
       'Cascais', 'Constância', 'Alcanena', 'Ferreira do Zêzere',
       'Santarém', 'Chaves', 'Boticas', 'Lamego', 'Montalegre',
       'Vila Real', 'Alijó', 'Murça', 'Vila Pouca de Aguiar', 'Valpaços',
       'Tarouca', 'Santa Marta de Penaguião', 'Vinhais',
       'Macedo de Cavaleiros', 'Freixo de Espada à Cinta',
       'Torre de Moncorvo', 'Mogadouro', 'Vimioso', 'Bragança',
       'Mirandela', 'Miranda do Douro', 'Moimenta da Beira', 'Monchique',
       'Reguengos de Monsaraz', 'Gouveia', 'Figueira de Castelo Rodrigo',
       'Manteigas', 'Mêda', 'Seia', 'Belmonte', 'Mação',
       'Oliveira de Frades', 'Nelas', 'Vouzela', 'São Pedro do Sul',
       'Albergaria-a-Velha', 'Santa Comba Dão', 'Mortágua', 'Tondela',
       'Vagos', 'Figueira da Foz', 'Alvaiázere', 'Soure',
       'Montemor-o-Velho', 'Valongo', 'Guimarães', 'Celorico de Basto',
       'Gondomar', 'Tomar', 'Sardoal', 'Rio Maior', 'Nazaré',
       'Caldas da Rainha', 'Ourém', 'Mafra', 'Peniche', 'Alenquer',
       'Torres Novas', 'Coruche', 'Paredes', 'Trofa', 'Portimão',
       'Silves', 'Loulé', 'Alcoutim', 'Vila Real de Santo António',
       'Barrancos', 'Castro Verde', 'Odemira', 'Ourique', 'Mora',
       'Portel', 'Ponte de Sor', 'Serpa', 'Sines', 'Moura',
       'Fornos de Algodres', 'Celorico da Beira', 'Ílhavo',
       'Condeixa-a-Nova', 'Barcelos', 'Arcos de Valdevez', 'Lousada',
       'Vila do Conde', 'Faro', 'Aljezur', 'Tavira', 'Lagoa', 'Albufeira',
       'Lagos', 'Olhão', 'São Brás de Alportel', 'Vila do Bispo',
       'Alter do Chão', 'Marvão', 'Gavião', 'Nisa', 'Castelo de Vide',
       'Sousel', 'Montemor-o-Novo', 'Cuba', 'Vendas Novas', 'Alvito',
       'Ferreira do Alentejo', 'Arraiolos', 'Portalegre', 'Alandroal',
       'Estremoz', 'Mourão', 'Pinhel', 'Sabugal', 'Vila Viçosa', 'Guarda',
       'Almeida', 'Penamacor', 'Castelo Branco', 'Vila Velha de Ródão',
       'Idanha-a-Nova', 'Aveiro', 'Oliveira do Bairro', 'Estarreja',
       'Sever do Vouga', 'Espinho', 'Batalha', 'Penafiel', 'Baião',
       'Felgueiras', 'Santo Tirso', 'Paços de Ferreira',
       'Castelo de Paiva', 'Vila Nova de Gaia', 'Oliveira de Azeméis',
       'Vale de Cambra', 'Vizela', 'Vila Verde', 'Póvoa de Lanhoso',
       'Esposende', 'Vila Nova de Cerveira', 'Caminha', 'Maia',
       'Póvoa de Varzim', 'Matosinhos', 'Cinfães', 'São João da Madeira',
       'Porto', 'Resende', 'Torres Vedras', 'Loures', 'Oeiras', 'Cadaval',
       'Vila Franca de Xira', 'Lisboa', 'Seixal', 'Lourinhã',
       'Sobral de Monte Agraço', 'Entroncamento', 'Montijo', 'Amadora',
       'Óbidos', 'Setúbal', 'Sesimbra', 'Palmela', 'Alpiarça', 'Moita',
       'Benavente', 'Vila Nova da Barquinha', 'Barreiro',
       'Arruda dos Vinhos', 'Almada', 'Bombarral', 'Alcobaça',
       'Alcochete', 'Peso da Régua', 'Sabrosa', 'Sernancelhe',
       'São João da Pesqueira', 'Penedono', 'Armamar', 'Mesão Frio',
       'Tabuaço', 'Vila Nova de Foz Côa', 'Carrazeda de Ansiães',
       'Vila Flor', 'Alfândega da Fé', 'Vidigueira', 'Castro Marim',
       'Campo Maior', 'Borba', 'Arronches', 'Odivelas', 'Golegã',
       'Monforte', 'Fronteira', 'Oliveira de Azeméis',
       'Vila Nova de Foz Côa', 'Castanheira de Pêra', 'Meda',
       'Freixo Espada à Cinta', 'Vila Velha de Ródão',
       'Sobral Monte Agraço']

In [59]:
for i in range(len(concelhos)):
    df.iloc[df[df['Concelho'] == raw[i]].index, 3] = concelhos[i]

In [60]:
df['Concelho'].value_counts()

Paredes                 9097
Penafiel                7847
Santa Maria da Feira    7781
Vila Nova de Gaia       7097
Gondomar                6575
                        ... 
Sousel                    13
Cuba                      11
Monforte                  11
Mourão                    11
Porto                      3
Name: Concelho, Length: 281, dtype: int64

### General formating and merging dfs

In [61]:
df['Ano'] = df['Ano'].astype('int32') 
df['Reacendimento'] = df['Reacendimento'].astype('int32')

In [62]:
df

,Ano,Codigo,Distrito,Concelho,DataAlerta,HoraAlerta,DataExtincao,HoraExtincao,AA_Total(pov+mato+agric),Reacendimento,Causa,TipoCausa,Grupo_causa,Categoria_causa,Dias_extincao
0,2001,BL20190,Coimbra,Oliveira do Hospital,02/07/2001,16:40:00,02/07/2001,17:25:00,0.3,1,12.0,negligente,desconhecida,desconhecida,0.0
1,2001,DM201114,Aveiro,Arouca,03/08/2001,17:47:00,06/08/2001,18:15:00,174.0,0,12.0,negligente,desconhecida,desconhecida,3.0
2,2001,DM401117,Viana do Castelo,Melgaço,17/09/2001,18:59:00,18/09/2001,02:40:00,54.0,0,12.0,negligente,desconhecida,desconhecida,1.0
3,2001,DM901112,Braga,Terras de Bouro,04/09/2001,23:05:00,05/09/2001,11:15:00,80.0,0,12.0,negligente,desconhecida,desconhecida,1.0
4,2001,DM901113,Braga,Amares,06/09/2001,18:50:00,07/09/2001,00:25:00,28.0,0,12.0,negligente,desconhecida,desconhecida,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321979,2015,RO315824,Setúbal,Almada,NaN,15:33:00,NaN,16:37:00,0.1,0,NaN,NaN,NaN,NaN,0.0
321980,2015,DM215306,Porto,Santo Tirso,NaN,16:44:00,NaN,20:31:00,2.0,0,NaN,NaN,NaN,NaN,0.0
321981,2015,DM415221,Braga,Barcelos,NaN,17:00:00,NaN,18:20:00,0.1,0,NaN,NaN,NaN,NaN,0.0
321982,2015,DM215434,Porto,Gondomar,NaN,17:57:00,NaN,19:48:00,0.1,0,NaN,NaN,NaN,NaN,0.0


In [63]:
def date_inverter(x): 
    if str(x) != 'nan': 
        return str(x).split('/')[-1] +'-'+ str(x).split('/')[-2] +'-'+ str(x).split('/')[-3]
    else: 
        return np.NaN

In [64]:
df.rename(columns={'Ano': 'year', 
                   'Codigo': 'code', 
                   'Distrito': 'region', 
                   'Concelho': 'county',
                   'DataAlerta': 'start_date', 
                   'HoraAlerta': 'start_time', 
                   'DataExtincao': 'end_date', 
                   'HoraExtincao': 'end_time', 
                   'AA_Total(pov+mato+agric)': 'total_ba',
                   'Reacendimento': 'rekindling',
                   'Causa': 'code_cause',
                   'TipoCausa': 'cause', 
                   'Grupo_causa': 'cause_group',
                   'Categoria_causa': 'cause_cat', 
                   'Dias_extincao': 'extinction_days'
                  }, inplace=True) 

In [65]:
#df.drop('Causa', axis=1, inplace=True)
df['start_date'] = df['start_date'].apply(lambda x: date_inverter(x)) 
df['end_date'] = df['end_date'].apply(lambda x: date_inverter(x))

In [66]:
df

,year,code,region,county,start_date,start_time,end_date,end_time,total_ba,rekindling,code_cause,cause,cause_group,cause_cat,extinction_days
0,2001,BL20190,Coimbra,Oliveira do Hospital,2001-07-02,16:40:00,2001-07-02,17:25:00,0.3,1,12.0,negligente,desconhecida,desconhecida,0.0
1,2001,DM201114,Aveiro,Arouca,2001-08-03,17:47:00,2001-08-06,18:15:00,174.0,0,12.0,negligente,desconhecida,desconhecida,3.0
2,2001,DM401117,Viana do Castelo,Melgaço,2001-09-17,18:59:00,2001-09-18,02:40:00,54.0,0,12.0,negligente,desconhecida,desconhecida,1.0
3,2001,DM901112,Braga,Terras de Bouro,2001-09-04,23:05:00,2001-09-05,11:15:00,80.0,0,12.0,negligente,desconhecida,desconhecida,1.0
4,2001,DM901113,Braga,Amares,2001-09-06,18:50:00,2001-09-07,00:25:00,28.0,0,12.0,negligente,desconhecida,desconhecida,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321979,2015,RO315824,Setúbal,Almada,NaN,15:33:00,NaN,16:37:00,0.1,0,NaN,NaN,NaN,NaN,0.0
321980,2015,DM215306,Porto,Santo Tirso,NaN,16:44:00,NaN,20:31:00,2.0,0,NaN,NaN,NaN,NaN,0.0
321981,2015,DM415221,Braga,Barcelos,NaN,17:00:00,NaN,18:20:00,0.1,0,NaN,NaN,NaN,NaN,0.0
321982,2015,DM215434,Porto,Gondomar,NaN,17:57:00,NaN,19:48:00,0.1,0,NaN,NaN,NaN,NaN,0.0


In [67]:
df_final = pd.concat([df, df2], axis=0, ignore_index=True) 
df_final.drop(df_final[df_final['total_ba'].isna()].index, inplace=True) 
df_final.reset_index(drop=True, inplace=True) 
df_final.drop(df_final[df_final['code'].isna()].index, inplace=True) 
df_final.iloc[329851:329853, 4:8] = np.NaN  
df_final.iloc[df_final[df_final['cause'] == 'nan'].index, 10] = np.NaN 
df_final.iloc[df_final[df_final['cause'] == 'null'].index, 10] = 'desconhecida' 
df_final.iloc[df_final[df_final['cause'] == ' '].index, 10] = 'desconhecida' 
df_final.drop(331466, inplace=True) 
df_final.drop(df_final[df_final['code'] == ' '].index, inplace=True) 
df_final.iloc[df_final[df_final['region'] == 'BRAGA'].index, 2] = 'Braga'

In [68]:
counties_to_parse = {'Almodóvar': 'Almodôvar', 
 'Freixo Espada à Cinta': 'Freixo de Espada à Cinta',
 'Montemor-o-novo': 'Montemor-o-Novo', 
 'Vila Vi?osa': 'Vila Viçosa', 
 'Alter do Ch?o': 'Alter do Chão', 
 'Ponte De Sor': 'Ponte de Sor', 
 'Vila Velha De R?d?o': 'Vila Velha de Ródão',
 'Idanha-a-nova': 'Idanha-a-Nova', 
 'Sert?': 'Sertã', 
 'Fund?o': 'Fundão', 
 'Proen?a-a-nova': 'Proença-a-Nova', 
 'Covilh?': 'Covilhã', 
 'Oliveira Do Hospital': 'Oliveira do Hospital', 
 'G?is': 'Góis', 
 '?lhavo': 'Ílhavo', 
 '?gueda': 'Águeda', 
 'Viana Do Castelo': 'Viana do Castelo', 
 'Paredes De Coura': 'Paredes de Coura', 
 'Arcos De Valdevez': 'Arcos de Valdevez',
 'Mon??o': 'Monção', 
 'Vila Nova de Cerveira': 'Vila Nova de Cerveira', 
 'VIZELA': 'Vizela', 
 'Our?m': 'Ourém', 
 'Ma??o': 'Mação',
 'Arruda Dos Vinhos': 'Arruda dos Vinhos', 
 'Gr?ndola': 'Grândola', 
 'Montemor-o-velho': 'Montemor-o-Velho', 
 'Cabeceiras De Basto': 'Cabeceiras de Basto', 
 'Oliveira Do Bairro': 'Oliveira do Bairro', 
 'Oliveira do Bairrro': 'Oliveira do Bairro', 
 'Figueira Da Foz': 'Figueira da Foz', 
 'Proença-a-nova': 'Proença-a-Nova', 
 'Condeixa-a-nova': 'Condeixa-a-Nova', 
 'Albergaria-a-velha': 'Albergaria-a-Velha', 
 'Freixo de Espada À Cinta': 'Freixo de Espada à Cinta',
 'M?rtola': 'Mértola', 
 'Almod?var': 'Almodôvar',
 'Mour?o': 'Mourão', 
 'Pa?os de Ferreira': 'Paços de Ferreira', 
 'Vila Do Conde': 'Vila do Conde', 
 'Bai?o': 'Baião', 
 'P?voa de Varzim': 'Póvoa de Varzim', 
 'Guimar?es': 'Guimarães', 
 'Vila Nova de Famalic?o': 'Vila Nova de Famalicão',                     
 'P?voa de Lanhoso': 'Póvoa de Lanhoso', 
 'P?voa De Lanhoso': 'Póvoa de Lanhoso', 
 'Gavi?o': 'Gavião', 
 'Marv?o': 'Marvão', 
 'Vila Velha de R?d?o': 'Vila Velha de Ródão', 
 'Alter Do Ch?o': 'Alter do Chão', 
 'Santa Comba D?o': 'Santa Comba Dão', 
 'S?o Pedro do Sul': 'São Pedro do Sul', 
 'S?o Pedro Do Sul': 'São Pedro do Sul'                
 }

In [69]:
for item in counties_to_parse.items():
    if len(df_final.iloc[df_final[df_final['county'] == item[0]].index, 3]) > 0:
        df_final.iloc[df_final[df_final['county'] == item[0]].index, 3] = item[1] 
    else: 
        continue

In [70]:
df_final

,year,code,region,county,start_date,start_time,end_date,end_time,total_ba,rekindling,code_cause,cause,cause_group,cause_cat,extinction_days
0,2001,BL20190,Coimbra,Oliveira do Hospital,2001-07-02,16:40:00,2001-07-02,17:25:00,0.300000,1.0,12.0,negligente,desconhecida,desconhecida,0.0
1,2001,DM201114,Aveiro,Arouca,2001-08-03,17:47:00,2001-08-06,18:15:00,174.000000,0.0,12.0,negligente,desconhecida,desconhecida,3.0
2,2001,DM401117,Viana do Castelo,Melgaço,2001-09-17,18:59:00,2001-09-18,02:40:00,54.000000,0.0,12.0,negligente,desconhecida,desconhecida,1.0
3,2001,DM901112,Braga,Terras de Bouro,2001-09-04,23:05:00,2001-09-05,11:15:00,80.000000,0.0,12.0,negligente,desconhecida,desconhecida,1.0
4,2001,DM901113,Braga,Amares,2001-09-06,18:50:00,2001-09-07,00:25:00,28.000000,0.0,12.0,negligente,desconhecida,desconhecida,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331655,2020,AG120614,NaN,NaN,2020-07-06,14:56:00,2020-07-07,10:00:00,535.632996,NaN,NaN,negligente,NaN,NaN,NaN
331656,2020,BL220206,NaN,NaN,2020-07-04,13:09:00,2020-07-05,00:53:00,156.279175,NaN,NaN,intencional,NaN,NaN,NaN
331657,2020,AT320158,NaN,NaN,2020-07-04,16:00:00,2020-07-04,17:07:00,1.407192,NaN,NaN,desconhecida,NaN,NaN,NaN
331658,2020,RO120327,NaN,NaN,2020-07-05,16:09:00,2020-07-05,20:39:00,26.590605,NaN,NaN,reacendimento,NaN,NaN,NaN


In [71]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331227 entries, 0 to 331659
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   year             331227 non-null  int32  
 1   code             331227 non-null  object 
 2   region           327591 non-null  object 
 3   county           327642 non-null  object 
 4   start_date       245355 non-null  object 
 5   start_time       331103 non-null  object 
 6   end_date         245160 non-null  object 
 7   end_time         330779 non-null  object 
 8   total_ba         331227 non-null  float64
 9   rekindling       321984 non-null  float64
 10  code_cause       107925 non-null  object 
 11  cause            116858 non-null  object 
 12  cause_group      101023 non-null  object 
 13  cause_cat        101023 non-null  object 
 14  extinction_days  321544 non-null  float64
dtypes: float64(3), int32(1), object(11)
memory usage: 39.2+ MB


In [72]:
df_final.to_csv(os.path.join(os.path.abspath(os.curdir), 'data', 'fire.csv'))